In [103]:
import cv2
#from google.colab.patches import cv2_imshow
import os
import numpy as np
import math
from PIL import Image
import PIL
import numpy as np
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.applications.imagenet_utils import preprocess_input,decode_predictions
from keras.applications.resnet50 import ResNet50
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
%matplotlib inline

In [37]:
video_path='video_captioning_dataset/mv89psg6zh4_33_46.avi'
s_path='video_captioning_dataset/frames/'
#source_directory='video_captioning_dataset/YouTubeClips/Eng_YouTubeClips'
# get the length of the video
def length_of_video(video_path):
    cap=cv2.VideoCapture(video_path)
    if (cap.isOpened()== False):
        print("Error opening video stream or file")
    length=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(length)
    return length

In [38]:
length=length_of_video(video_path)

In [39]:
frame_path=s_path+'1'+'.jpg'
frame_path

'video_captioning_dataset/frames/1.jpg'

In [34]:
#def extracting_frames(video_path,save_path,skip_frames=20):
 #   print("5")
# Extracting frames using the absolute difference between the pixels of the two image arrays
# it gives the pixels of the objects taht are moving
#absdiff is a 3 dim array
p_frame_thresh=200000
length=length_of_video(video_path)
cap=cv2.VideoCapture(video_path) 
count=1
ret,prev_frame=cap.read()
    #if success and count%skip_frames==0:
while ret:
    ret,curr_frame=cap.read()
    if ret:
        count+=1
        frame_path=s_path+str(count)+'.jpg'
        diff=cv2.absdiff(curr_frame,prev_frame)
        non_zero_count=np.count_nonzero(diff)
        if non_zero_count>p_frame_thresh:
            #print("got P-Frame")
            cv2.imwrite(frame_path,prev_frame)
        prev_frame=curr_frame    
    
#frame_path=s_path+str(count)+'.jpg'
#cv2.imwrite(frame_path,frame)
#print(frame.shape)
      #  count+=1
   # print(frame_path)
    #else:
    #    count+=1
cap.release()    
        
   

got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame
got P-Frame


In [30]:
# using Frame rate to extract key frames
cap = cv2.VideoCapture(video_path)
save_folder='video_captioning_dataset/framerate_images/'
frameRate=cap.get(5)
count=1
while (cap.isOpened()):
    frameid=cap.get(1)# current frame number
    ret,frame=cap.read()
    if (ret!=True):
        break
    if (frameid%math.floor(frameRate)==0):
        filename=save_folder+str(count)+'.jpg'
        cv2.imwrite(filename,frame)
        count+=1
cap.release()

In [91]:
##using mean shift to extract key frames
cap = cv2.VideoCapture(video_path)

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
r,h,c,w = 250,90,400,125  # simply hardcoded the values
track_window = (c,r,w,h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        #cv2.imshow('img2',img2)
         
        k = cv2.waitKey(60) & 0xff
        #print(k)
        if k == 27:
            break
        else:
            #print(chr(k))
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

cv2.destroyAllWindows()
cap.release()

In [ ]:
#video_path='video_captioning_dataset/mv89psg6zh4_33_46.avi'
#loop through all the video clips and generate key frames for each
save_folder='video_captioning_dataset/YouTubeClips/Training_clips_keyframes/'
source_directory='video_captioning_dataset/YouTubeClips/Training_clips'
# get the length of the video
for filename in os.listdir(source_directory):
    #print(filename)
    if not filename.endswith('avi'):
        break
    else:
        video_path=os.path.join(source_directory, filename)
        cap = cv2.VideoCapture(video_path)
        frameRate=cap.get(5)
        count=1
        nfilename=filename.split('.')[0]
        #print(nfilename)
        while (cap.isOpened()):
            frameid=cap.get(1)# current frame number
            ret,frame=cap.read()
            #print(ret,count)
            if (ret!=True):
                break
            if (frameid%math.floor(frameRate)==0):
                sfilename=save_folder+nfilename+'_'+'keyframe'+str(count)+'.jpg'
                cv2.imwrite(sfilename,frame)
                count+=1
    
cap.release()

In [41]:
# get the count of total keyframes
path,dirs,files = next(os.walk('video_captioning_dataset/YouTubeClips/Training_clips_keyframes'))
file_counts = len(files)
#type(files)
print(file_counts)
test_img=files[0]


16707


In [96]:
from keras.preprocessing.image import ImageDataGenerator
train_path='video_captioning_dataset/YouTubeClips/Training_clips_keyframes'
#Valid_path='video_captioning_dataset/YouTubeClips/Training_clips_keyframes'
#Test_path='video_captioning_dataset/YouTubeClips/Training_clips_keyframes'

train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),batch_size=10)

Found 0 images belonging to 0 classes.


In [98]:
images=next(train_batches)

In [104]:
#plots(images)

NameError: name 'plots' is not defined

In [152]:
def preprocess_image(path):
    #Load the images from the given file path and resize it
    print(path)
    img=Image.open(path)
    new_size=(224,224)
    width=224
    height=224
    img=img.resize(size=new_size)
    
    # convert the image to numpy array
    img=np.array(img).reshape(-1,width,height,1)
    # scale the image-pixels so they fall between 0 to 1
    img=img/255.0
    
    # convert the 2-dim array to 3-dim RGB array
    img=np.repeat(img[:,:,np.newaxis],3,axis=2)
    return img


In [153]:
# Image preprocessing using Resnet50
image_model=ResNet50(weights='imagenet',input_shape=(224,224,3))

In [164]:
#image_model.summary()

In [155]:
# create a model removing the last layer
transfer_layer=model.layers[-2]

In [156]:
model_transfer=Model(inputs=model.input,outputs=transfer_layer.output)

In [157]:
transfer_values_size=K.int_shape(transfer_layer.output)[1]
transfer_values_size

2048

In [159]:
#model_transfer.summary()

In [160]:

#img_size=[224,224,3]
#source_directory='video_captioning_dataset/YouTubeClips/Training_clips_keyframes'
#video_path=os.path.join(source_directory, "0bSz70pYAP0_5_15_keyframe1.jpg")
#img=PIL.Image.open(video_path).convert("L")
#newsize=[224,224]
#img=img.resize(newsize)
#img_arr=np.array(img)
#test_arr=preprocess_image(video_path)
#test_arr

In [161]:
def process_images(dir,filenames,batch_size=30):
    #process all the files in the given directory
    # we process images in batches to save memory and improve efficiency on the GPU
    
    img_size=(224,224)
    #Number of images to process
    num_images=len(filenames)
    # pre-allocate input batch array
    # input array
    im_shape=(batch_size,)+img_size+(3,)
    image_batch=np.zeros(shape=im_shape,dtype=np.float16)
    
    # output array for transfer values
    tv_shape=(num_images,transfer_values_size)
    transfer_values=np.zeros(shape=tv_shape,dtype=np.float16)
    
    # Initialize index into the filename
    start_index=0
    #process batches of image-files
    while start_index<num_images:
        #print_progress(count=start_index,max_count=num_images)
        
        end_index=start_index+batch_size
        if end_index>num_images:
            end_index=num_images
            
        current_batch_size=end_index -start_index  
        # Load all the images in the batch
        for i,filename in enumerate(filenames[start_index:end_index]):
            path=os.path.join(dir,filename)# path of the filename
            image_p=preprocess_image(path)
            image_batch[i]=image_p
        transfer_vales_batch=\
            model_transfer.predict(image_batch[0:current_batch_size])
        transfer_values[start_index:end_index]=\
            transfer_values_batch[0:current_batch_size]
        
        start_index=end_index
        
    return transfer_values        
            
        
    
    

In [162]:
import csv
def process_images_train():
    
    filename='transfervalues_train.csv'
    transfer_values= process_images(dir="video_captioning_dataset/YouTubeClips",filenames="filenames_train")
    with open(filename,'w') as csvfile:
        csvwriter=csv.writer(transfer_values)
     #f.write("%s,%s\n"%(key,training_captions[key]))
#return transfer_values       

In [163]:
process_images_train()

video_captioning_dataset/YouTubeClips\f


FileNotFoundError: [Errno 2] No such file or directory: 'video_captioning_dataset/YouTubeClips\\f'